In [1]:
import pandas as pd
import talib as ta
import os
from ftx_client import FtxClient
import supertrend as spt
import backtesting as bt
from config import API_KEY, API_SECRET
from tqdm import tqdm

In [2]:
BACKTEST_FOLDER = "./backtest"
if not os.path.exists(BACKTEST_FOLDER):
    os.makedirs(BACKTEST_FOLDER)

ANALYSIS_FILEPATH = os.path.join(BACKTEST_FOLDER, "MarketAnalysis_40days_median.csv")
OPTIMIZEDML_FILEPATH = os.path.join(BACKTEST_FOLDER, "OptimizedML_40days_median.csv")

In [3]:
ftx = FtxClient(api_key=API_KEY, api_secret=API_SECRET)

In [4]:
markets = []
for future in ftx.list_futures():
    if future["type"] == "perpetual":
        if future["volumeUsd24h"] > 1e7:
            markets.append(future["name"])

In [5]:
analysis_df = pd.DataFrame(
    columns=["Name", "AvgReturns", "StdDev", "RetDevRatio", "MinReturns", "MaxReturns",
             "AvgNegReturns", "AvgPosReturns", "PosNegRetRatio", "MedReturns", "MedNegReturns",
             "MedPosReturns", "MedPosNegRetRatio"])
optimzed_ml = pd.read_csv(OPTIMIZEDML_FILEPATH)

for market in tqdm(markets):
    df = ftx.get_historical_market_data(market, interval="4h", start_time="40 days ago")
    
    if len(df) < 240:
        continue
        
    multiplier = optimzed_ml.loc[optimzed_ml['Name'] == market]["Multiplier"].values[0]
    lookback = optimzed_ml.loc[optimzed_ml['Name'] == market]["Lookback"].values[0]
    
    profits = bt.backtest_dataframe(df, look_back=lookback, multiplier=multiplier)
    profits["Name"] = market
    analysis_df = analysis_df.append(profits, ignore_index=True)

100%|██████████| 36/36 [00:56<00:00,  1.56s/it]


In [6]:
analysis_df = analysis_df.round(1)
analysis_df.to_csv(ANALYSIS_FILEPATH, index=False)

In [7]:
pd.read_csv(ANALYSIS_FILEPATH)

,Name,AvgReturns,StdDev,RetDevRatio,MinReturns,MaxReturns,AvgNegReturns,AvgPosReturns,PosNegRetRatio,MedReturns,MedNegReturns,MedPosReturns,MedPosNegRetRatio
0,AAVE-PERP,10.5,18.2,0.6,-11.4,46.7,-4.0,22.5,5.6,0.2,-2.5,26.5,10.4
1,ADA-PERP,11.7,8.7,1.3,-0.3,20.2,-0.3,17.6,65.3,15.1,-0.3,17.6,65.3
2,ATOM-PERP,3.6,10.4,0.3,-6.8,22.7,-3.9,13.0,3.4,-0.6,-3.1,13.4,4.3
3,AUDIO-PERP,10.1,18.8,0.5,-9.2,67.5,-3.2,18.4,5.7,5.2,-1.4,12.5,8.9
4,AXS-PERP,110.3,203.8,0.5,-3.8,517.3,-2.7,185.7,68.4,11.8,-2.7,28.0,10.3
5,BCH-PERP,12.0,12.0,1.0,-3.2,26.2,-3.2,19.6,6.1,12.9,-3.2,19.6,6.1
6,BNB-PERP,4.0,6.7,0.6,-5.2,21.4,-1.5,7.6,5.3,1.5,-0.8,6.0,7.2
7,BTC-PERP,4.7,8.0,0.6,-1.4,24.3,-0.9,7.5,7.9,0.6,-0.8,3.9,4.7
8,CHZ-PERP,5.4,7.8,0.7,-2.8,22.8,-1.5,8.1,5.3,2.3,-1.5,5.9,3.8
9,COMP-PERP,9.8,15.3,0.6,-4.5,50.6,-2.0,19.9,9.8,7.3,-1.9,16.0,8.5
